## Clustering cells in different groups and visualize the enrichment of each category in each cluster


#### Steps:

- will be added (to do)
  

In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(color_codes=True)
import matplotlib.pyplot as plt
import pickle
import os
from sklearn.cluster import KMeans
from singlecell.read import read_single_cell_sql
from singlecell.process import extract_single_cell_samples
from singlecell.visualize import visualize_n_SingleCell, cluster
from singlecell.process.normalize_funcs import *
from singlecell.preprocess import filter_out_edge_single_cells, extract_cpfeature_names,handle_nans
import time

In [ ]:
# ls /home/ubuntu/bucket/projects/2016_08_01_RadialMitochondriaDistribution_donna/workspace/drugSCprofiles/

### Input parameters

In [6]:
images_dir="/home/ubuntu/bucket/projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/2016_04_01_a549_48hr_batch1_compressed/"
res_dir="./"

controlLabel='control'
drugLabel='drugX'


In [ ]:
# ls /home/ubuntu/bucket/projects/2016_08_01_RadialMitochondriaDistribution_donna/workspace/drugSCprofiles

### Reading two single cell data dataframes for control and pertX
    * This is an example of two single cell dataframe, you may ignore below cell if you have your own inputs

In [7]:
fileName="/home/ubuntu/bucket/projects/2016_08_01_RadialMitochondriaDistribution_donna/workspace/drugSCprofiles/BRD-A00147595-001-01-5_0.12.pkl"    
pair_dict = pickle.load( open(fileName, "rb" ) )
df_sc_pert=pair_dict['cp_ss']
df_sc_control=pair_dict['cp_ss_control']

df_sc_pert["label"]=drugLabel
df_sc_control["label"]="control"

sc_df = pd.concat([df_sc_pert,df_sc_control],ignore_index=True)
sc_df,_ = filter_out_edge_single_cells.edgeCellFilter(sc_df,edge_margin=50);

cp_features, cp_features_analysis_0 =  extract_cpfeature_names.extract_cpfeature_names(sc_df);
sc_df, cp_features_analysis = handle_nans.handle_nans(sc_df,cp_features_analysis_0,fill_na_method='drop-rows');   


image_width 2160
cp_features: 887
len cp_features_analysis/nan cols/low vars: 877 0 10
before dropping nan rows:  (8264, 1092)
after dropping nan rows:  (8264, 1092)


In [6]:
# standardized_sc_df['Nuclei_Location_Center_X']

### Creating "PathName_Orig"+channelX column for input dataframe containing the correct location of images

In [8]:
images_dir="/home/ubuntu/bucket/projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/2016_04_01_a549_48hr_batch1_compressed/"

channels=["Mito","AGP","DNA","RNA","ER"]
for ch in channels:
    sc_df["PathName_Orig"+ch]=images_dir+"images/"+sc_df["Metadata_Plate"]
    sc_df["FileName_Orig"+ch]=sc_df["FileName_Orig"+ch].apply(lambda x: x.replace("tiff","png"))


In [5]:
sc_df["PathName_Orig"+ch].values[0]

'/home/ubuntu/bucket/projects/2015_10_05_DrugRepurposing_AravindSubramanian_GolubLab_Broad/2016_04_01_a549_48hr_batch1_compressed/images/SQ00015196'

### Apply Clustering Subpopulation analysis on input data
    And:
    - save the resulting images in "res_dir" path

In [22]:
%%capture
os.system("mkdir -p "+res_dir+controlLabel+'-'+drugLabel)

#### standardize cp feature columns
feature_scaler='Standard'
standardized_sc_df = standardize_df_columns(sc_df,cp_features_analysis,feature_scaler)


nSampleSCs=6;
boxSize=100;
n_clusters=10

standardized_sc_df=cluster.add_clustering_index_column(standardized_sc_df,cp_features_analysis,'kmeans',n_clusters);
    
params={'results_dir':res_dir+controlLabel+'-'+drugLabel,\
        'control_label':controlLabel,\
        'pert_label': drugLabel,\
        'n_clusters':n_clusters,\
        'n_cells_4single_cell_plots':nSampleSCs,\
        'channels_4single_cell_plots':channels,\
        'boxSize_4single_cell_plots': boxSize,\
        'compressed_ims_flag':True,\
        'compressed_im_size':1080,\
        'img_save_format':'.png'}     

cluster.subpopulation_clustering_visualization(standardized_sc_df,cp_features_analysis,params)


In [25]:
1/(20*100)

0.0005

In [18]:
histDiff=[-0.0136275,0.01224709,0.01980404,0.00794258,-0.00012969,-0.02341193,\
-0.01126656,-0.00807546,0.00958956,0.00692786]


sortedDiff=[-0.02341193,-0.0136275, -0.01126656,-0.00807546,-0.00012969 ,0.00692786,\
0.00794258,0.00958956,0.01224709,0.01980404]
# [0.11226907 0.07026686 0.17827254 0.07737249 0.02731723 0.0571609
#  0.1555345  0.08416232 0.0723196  0.16532449]

# histDiff [-0.0136275   0.01224709  0.01980404  0.00794258 -0.00012969 -0.02341193
#  -0.01126656 -0.00807546  0.00958956  0.00692786]
# sortedDiff [-0.02341193 -0.0136275  -0.01126656 -0.00807546 -0.00012969  0.00692786
#   0.00794258  0.00958956  0.01224709  0.01980404]
# [1, 8, 9, 6, 4, 0, 2, 3, 7, 5]

In [19]:
np.sum(histDiff)

-9.99999999907325e-09

In [36]:
# map_4ordered_clusters={}
map_4ordered_clusters=dict(zip(np.argsort(histDiff), range(10)))
map_4ordered_clusters

{5: 0, 0: 1, 6: 2, 7: 3, 4: 4, 9: 5, 3: 6, 8: 7, 1: 8, 2: 9}

In [33]:
list(np.argsort(histDiff))[0]

5

In [23]:
# np.searchsorted([1,2,3,4,5], [-10, 10, 2, 3])
# i

for 

9

In [21]:
# pd.read_csv(gi).shape
ind=[]
for i in range(len(histDiff)):
    iinndd = np.where(histDiff[i]==sortedDiff)[0].tolist()
    for j in range(len(iinndd)):
        if iinndd[j] not in ind:
            ind=ind+[iinndd[j]]
            break
            
print(ind)

[]


In [17]:
iinndd

[]

In [16]:
iinndd

[]

## Subpopulation analysis for pooled data

In [ ]:
control_guides.index('TTTTACCTTGTTCACATGGA')

In [ ]:
import glob
rootDir='/home/ubuntu/calbucket/projects/2018_11_20_Periscope_Calico/'

# batch='20200805_A549_WG_Screen';
batch='20210422_6W_CP257';


batch_multi_name_dict={'20210422_6W_CP257':'CP257-HeLa-WG',\
                   '20200805_A549_WG_Screen':'CP186-A549-WG'}
batchName2=batch_multi_name_dict[batch]
sc_files_dir=rootDir+'workspace/software/'+batchName2+'/data/1.profiles/'+batch+'/single_cell/single_cell_by_guide/'
im_size=5500 # hardcoded for now, TODO: create a dictionary if this number is different for 257 vs 186

### metadata 
metadata_dir=rootDir+'workspace/metadata/'+batch+'/'
metadata_orig= pd.read_csv(metadata_dir+'Barcodes.csv')

# input_gene='KRT28'
# n_cells=20
box_size=100

# how we select cells which can be 'random','representative','geometric_median'
cell_selection_method='representative'
channels=['DNA','Mito','Phalloidin','WGA','ER','Outline']

resultsDir='/home/ubuntu/bucket/projects/2018_11_20_Periscope_Calico/workspace/visualizations/'+batch+\
'/subpopulations/'

#########################
# genes_ls=metadata_orig.gene_symbol.unique().tolist()
control_vals='nontargeting'
control_guides=metadata_orig[metadata_orig['gene_symbol']==control_vals].sgRNA.unique().tolist()

control_guide=control_guides[10]
control_file=batch+'_single_cell_normalized_ALLBATCHES__'+control_guide+'_'+control_vals+'.csv.gz'
df_control=pd.read_csv(sc_files_dir+control_file);
cell_selection_method='random'
df_control_subset,cp_features_control = extract_single_cell_samples(df_control,100,cell_selection_method);
df_control_subset['label']='control'
############################
box_size=100
im_size=5500
n_cells=0
cell_selection_method='representative'
channels=['DNA','Mito','Phalloidin','WGA','ER','Outline']

# genes_ls=metadata_orig.gene_symbol.unique().tolist()
# genes_ls=['SFPQ','ITGB5','ELOB','C16orf72','TNPO1','PPP1R3E','NLE1','PSMD3','PRC1',\
# 'NIPBL','DYNC1I2','ILK','LOC102724813','MCM7','FBL','NPIPA2','PSMA6','PSMA5','DCTN5','HSPE1','STIL','ATP6V0B']
# genes_ls=['ATP6V0D1']
genes_ls=['TOMM20']

for igi in range(len(genes_ls)):
    input_gene=genes_ls[igi]

    gene_guids_ls=glob.glob(sc_files_dir+'*_'+input_gene+'.csv.gz')
#     sfdss
    for gi in gene_guids_ls:
        df_p_s=pd.read_csv(gi);
#         channels=['DNA','Mito','Phalloidin','WGA','ER','Outline']
#         for ch in channels:
#             df_p_s["PathName_Corr"+ch]=rootDir+batch+'/images_corrected_cropped/'+df_p_s["Metadata_Foci_plate"]+'_'+df_p_s["Metadata_Foci_well"]+'/Corr'+ch
#             df_p_s["FileName_Corr"+ch]="Corr"+ch+"_"+"Site_"+df_p_s["Metadata_Foci_site_location"].astype(str)+".tiff"

#         df_p_s["Path_Outlines"]=rootDir+'workspace/analysis/'+batch+'/'+df_p_s["Metadata_Foci_plate"]+'-'+df_p_s["Metadata_Foci_well"]+'-'+df_p_s["Metadata_Foci_site_location"].astype(str)+'/'\
#         +'/CorrDNA_Site_'+df_p_s["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

#         df_p_s["Nuclei_Location_Center_X"]=df_p_s["Cells_AreaShape_Center_X"];
#         df_p_s["Nuclei_Location_Center_Y"]=df_p_s["Cells_AreaShape_Center_Y"];
        
        
        
        if df_p_s.shape[0]>100:
            n_cells=100
            df_samples,cp_features_pert = extract_single_cell_samples(df_p_s,n_cells,cell_selection_method);
            cp_features_analysis=list(set(cp_features_control) & set(cp_features_pert))
            df_samples['label']=input_gene
            df_sc=pd.concat([df_samples,df_control_subset],join='inner',ignore_index=True).drop_duplicates(ignore_index=True)
            resdir=resultsDir+input_gene+'/'+gi.split('_')[-2]
#             os.system("mkdir -p "+resdir)
#             fgsdfd
            for ch in channels:
                df_sc["PathName_Corr"+ch]=rootDir+batch+'/images_corrected_cropped/'+df_sc["Metadata_Foci_plate"]+'_'+df_sc["Metadata_Foci_well"]+'/Corr'+ch
                df_sc["FileName_Corr"+ch]="Corr"+ch+"_"+"Site_"+df_sc["Metadata_Foci_site_location"].astype(str)+".tiff"

            df_sc["Path_Outlines"]=rootDir+'workspace/analysis/'+batch+'/'+df_sc["Metadata_Foci_plate"]+'-'+df_sc["Metadata_Foci_well"]+'-'+df_sc["Metadata_Foci_site_location"].astype(str)+'/'\
            +'/CorrDNA_Site_'+df_sc["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

            df_sc["Nuclei_Location_Center_X"]=df_sc["Cells_AreaShape_Center_X"];
            df_sc["Nuclei_Location_Center_Y"]=df_sc["Cells_AreaShape_Center_Y"];

            df_sc=edgeCellFilter2(df_sc,im_size,box_size/2);
            
            ################################
            nClus=20
            kmeans = KMeans(n_clusters=nClus, random_state=0).fit(df_sc[cp_features_analysis].values)
            clusterLabels=kmeans.labels_
            distances=kmeans.fit_transform(df_sc[cp_features_analysis].values)
            df_sc['clusterLabels']=clusterLabels;
            df_sc['dist2Mean']=np.min(distances,1);

            DirsDict={}
            # DirsDict['root']=rootDir
            DirsDict['resDir']=resdir

            clusteringHists(DirsDict,df_sc,'control',input_gene,nClus,cp_features_analysis,channels,box_size,pooled=True)
            plt.close('all')
            
        else:
            print(df_p_s.shape[0], " number of single cells is less than the threshold for doing the analysis!")

In [ ]:
# df_control.columns[df_control.columns.str.contains("Align_Xshift_")].tolist()
# gene_guids_ls
# input_gene
# df_sc=
# pd.concat([df_samples,df_control_subset], join='inner',ignore_index=True).drop_duplicates(ignore_index=True)
# df_control_subset
# df_sc
# clusteringHists(DirsDict,df_sc,'control',input_gene,nClus,cp_features_analysis,channels,box_size,pooled=True)
# igi
gene_guids_ls

In [ ]:
gene_guids_ls

In [ ]:
# df_control_subset.shape
# df_sc['label']
# input_gene
# df_sc
# sc_files_dir+'*_'+input_gene+'.csv.gz'
# for ch in channels:
#     print(df_sc["PathName_Corr"+ch].values)
# pd.concat([df_samples,df_control_subset],ignore_index=True).drop_duplicates(ignore_index=True).shape
df_sc.shape

In [ ]:
input_gene

In [ ]:
df_p_s.shape

In [ ]:
rootDir+'workspace/analysis/'+batch+'/'+df_samples["Metadata_Foci_plate"]+'-'+df_samples["Metadata_Foci_well"]+'-'+df_samples["Metadata_Foci_site_location"].astype(str)+'/'\
        +'/CorrDNA_Site_'+df_samples["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

In [ ]:
rootDir+'workspace/analysis/'+batch+'/'+df_control_subset["Metadata_Foci_plate"]+'-'+df_control_subset["Metadata_Foci_well"]+'-'+df_control_subset["Metadata_Foci_site_location"].astype(str)+'/'\
        +'/CorrDNA_Site_'+df_control_subset["Metadata_Foci_site_location"].astype(str)+'_Overlay.png'

In [ ]:
df_control_subset.shape,df_p_s.shape

In [ ]:
# 20200805_A549_WG_Screen_single_cell_normalized_ALLBATCHES__TTCTTAGAAGTTGCTCCACG_nontargeting.csv.gz
# 20200805_A549_WG_Screen_single_cell_normalized_ALLBATCHES__TTCTTAGAAGTTGCTCCACG_nontargeting.csv.gz
# metadata_orig[metadata_orig['gene_symbol']=='nontargeting'].sgRNA.unique()
# gene_guids_ls=glob.glob(sc_files_dir+'*_nontargeting.csv.gz')
# 20200805_A549_WG_Screen_single_cell_normalized_ALLBATCHES__TTCTTAGAAGTTGCTCCACG_nontargeting.csv

In [ ]:
with open('./utils/blackListFeatures.pkl', 'rb') as f:
    blackListFeatures = pickle.load(f)

In [ ]:
mynewlist